In [1]:
import os,nbloader,warnings,sys,time
warnings.filterwarnings("ignore")
from demo_cn_cls import cn_cls_class
import torchvision.transforms as transforms
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from util import create_gradient_clipping,gpusession,print_n_txt,augment_img,grid_maker,get_properIdx
from data.cifar import CIFAR10
from multiprocessing import Process,Manager
print ("Packages loaded")

Packages loaded


### Configuration

In [2]:
lr_list = [1e-1] # not used, lr is based on optm
pi1_bias_list = [2e-1]
k_mix_list = [10]
USE_GAP_list = [True]
mixup_val_list = [1,4,8,16] # [0,1,16]
OPTM_list = ['sgd']# ['adam','sgd']
run_mode_list = [0,1,2] # [0:pairflip-0.45, 1:symmetric-0.2, 2:symmetric-0.5]
max_epoch_list = [200]
logsumexp_coef_list = [1e-4]

In [3]:
def get_reg_config(_processID=0,_maxProcessID=8,_maxGPU=8):
    # Get total configurations
    _G = grid_maker(lr_list,pi1_bias_list,k_mix_list,USE_GAP_list,mixup_val_list,OPTM_list,run_mode_list,max_epoch_list,logsumexp_coef_list)
    _ids = get_properIdx(_processID,_maxProcessID,_nTask=_G.nIter)
    _paramsList = list(_G.paramList[i] for i in _ids) 
    # Get corresponding GPU ID 
    _GPU_ID = (_processID % _maxGPU)
    return _paramsList,_GPU_ID

In [4]:
def train_wrapper(_paramsList,_GPU_ID):
    for pIdx,params in enumerate(_paramsList): # For all current configurations
        print ("===[%d/%d]==="%(pIdx,len(_paramsList)))
        _lr,_pi1_bias,_k_mix,_USE_GAP,mixup_val,_optm,_run_mode,_max_epoch,_logsumexp_coef = \
            params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8]
        
        # Get Data
        if _run_mode == 0:
            noise_type = 'pairflip' # [pairflip, symmetric]
            noise_rate = 0.45
        elif _run_mode == 1:
            noise_type = 'symmetric' # [pairflip, symmetric]
            noise_rate = 0.2
        elif _run_mode == 2:
            noise_type = 'symmetric' # [pairflip, symmetric]
            noise_rate = 0.5
        train_dataset = CIFAR10(root='./data/',download=True,train=True,transform=transforms.ToTensor(),
                        noise_type=noise_type,noise_rate=noise_rate)
        test_dataset = CIFAR10(root='./data/',download=True,train=False, transform=transforms.ToTensor(),
                               noise_type=noise_type,noise_rate=noise_rate)
        def one_hot(a, num_classes): # to 1-hot vectors
            return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
        x_train = train_dataset.train_data/255. # rgb values between 0~1
        n_train = np.shape(x_train)[0]
        x_train = x_train.reshape([n_train,-1])
        y_train = one_hot(train_dataset.train_labels,10)
        y_noisy_train = one_hot(np.asarray(train_dataset.train_noisy_labels),10)
        x_test = test_dataset.test_data/255. # rgb values between 0~1
        n_test = np.shape(x_test)[0]
        x_test = x_test.reshape([n_test,-1])
        y_test = one_hot(np.asarray(test_dataset.test_labels),10)
        label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        # Train
        lr = _lr
        pi1_bias = _pi1_bias
        k_mix = _k_mix
        USE_GAP = _USE_GAP
        if _optm == 'sgd': 
            USE_SGD = True
            lr = 1e-2
        elif _optm == 'adam': 
            USE_SGD = False
            lr = 1e-3
        if USE_GAP: gap_str = 'gap'
        else: gap_str = 'dense'
        if mixup_val == 0:
            USE_MIXUP = False
            mix_str = 'nomix'
        else:
            USE_MIXUP = True
            mix_str = 'mixup%02d'%(mixup_val)
        max_epoch = _max_epoch
        logsumexp_coef = _logsumexp_coef
        GPU_ID = _GPU_ID
        
        def leaky_relu(x):
            return tf.nn.leaky_relu(x,alpha=0.01)
        CN = cn_cls_class(_name='cn_cls_cifar10_%s_rate%d_lr%.0e_pi1_bias%.0e_k_mix%d_%s_%s_%s_max_epoch%d_lsecoer%.0e'%
                                  (noise_type,noise_rate*100,lr,pi1_bias,k_mix,gap_str,mix_str,_optm,max_epoch,logsumexp_coef),
                          _xdim=[32,32,3],_ydim=10,
                          _hdims=[128,128,128,256,256,256,512,256,128],_filterSizes=[3,3,3,3,3,3,3,3,3],
                          _max_pools=[1,1,2,1,1,2,1,1,2],_feat_dim=128,
                          _kmix=k_mix,_actv=leaky_relu,_bn=slim.batch_norm,
                          _rho_ref_train=0.95,_tau_inv=1e-4,_pi1_bias=pi1_bias,_logSigmaZval=-2,
                          _logsumexp_coef=logsumexp_coef,_kl_reg_coef=1e-4,_l2_reg_coef=1e-6,
                          _momentum = 0.9,
                          _USE_INPUT_BN=False,_USE_RESNET=False,_USE_GAP=USE_GAP,_USE_KENDALL_LOSS=False,_USE_SGD=USE_SGD,
                          _USE_MIXUP=USE_MIXUP,_mixup_val=mixup_val,
                          _GPU_ID=GPU_ID,_VERBOSE=0)
        tf.set_random_seed(0)
        np.random.seed(0)
        sess = gpusession(); sess.run(tf.global_variables_initializer()) 
        CN.train(_sess=sess,
                 _trainimg=x_train,_trainlabel=y_noisy_train,
                 _testimg=x_test,_testlabel=y_test,
                 _valimg=x_test,_vallabel=y_test,
                 _maxEpoch=max_epoch,_batchSize=128,_lr=lr,_LR_SCHEDULE=True,_kp=1.0,_PRINT_EVERY=100,_SAVE_BEST=False,
                 _DO_AUGMENTATION=True,_VERBOSE_TRAIN=True)
        sess.close()

In [5]:
class worker_class(Process):
    def __init__(self,_idx=0,_maxProcessID=8,_maxGPU=8,_name='worker',_VERBOSE=True):
        Process.__init__(self)
        # Initialize class
        self.idx = _idx
        self.maxProcessID = _maxProcessID
        self.maxGPU = _maxGPU
        self.name = _name
        self.VERBOSE = _VERBOSE
        # Initialize Process
        self.setName = 'T_'+self.name
        self.daemon = True # Make sure that each child is killed when the parent is dead. (?!)
        # Print
        if self.VERBOSE:
            print ("[%s] Instantiated."%(self.name))
    def run(self):
        print ("Starting [%s]"%(self.name))
        time.sleep(1e-1)
        # Get configurations 
        paramsList,GPU_ID = get_reg_config(self.idx,self.maxProcessID,self.maxGPU)
        print ("processID:[%d/%d] GPU_ID:[%d] #Config:[%d]"
               %(self.idx,self.maxProcessID,GPU_ID,len(paramsList)))
        # Run
        train_wrapper(_paramsList=paramsList,_GPU_ID=GPU_ID)
        print ("[%s] Finished."%(self.name))

### Run

In [6]:
nWorker = 8
maxGPU  = 4
WORKERS = ['']*nWorker 
for i in range(nWorker):
    WORKERS[i] = worker_class(_idx=i,_maxProcessID=nWorker,_maxGPU=maxGPU
                              ,_name='Worker_%02d'%(i),_VERBOSE=False)
    print ("[%d/%d] worker initiated."%(i,nWorker))
for i in range(nWorker):
    WORKERS[i].start(); # Start process

[0/8] worker initiated.
[1/8] worker initiated.
[2/8] worker initiated.
[3/8] worker initiated.
[4/8] worker initiated.
[5/8] worker initiated.
[6/8] worker initiated.
[7/8] worker initiated.
Starting [Worker_01]
Starting [Worker_02]
Starting [Worker_00]
Starting [Worker_03]
Starting [Worker_04]
Starting [Worker_05]
Starting [Worker_06]
Starting [Worker_07]
processID:[1/8] GPU_ID:[1] #Config:[2]
===[0/2]===
processID:[2/8] GPU_ID:[2] #Config:[2]
===[0/2]===
processID:[0/8] GPU_ID:[0] #Config:[2]
processID:[3/8] GPU_ID:[3] #Config:[2]
===[0/2]===
===[0/2]===
processID:[4/8] GPU_ID:[0] #Config:[1]
===[0/1]===
processID:[5/8] GPU_ID:[1] #Config:[1]
===[0/1]===
processID:[6/8] GPU_ID:[2] #Config:[1]
===[0/1]===
processID:[7/8] GPU_ID:[3] #Config:[1]
===[0/1]===
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
File

 [0.45 0.   0.   0.   0.   0.   0.   0.   0.   0.55]]
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Text name: res/res_cn_cls_cifar10_symmetric_rate50_lr1e-02_pi1_bias2e-01_k_mix10_gap_mixup01_sgd_max_epoch200_lsecoer1e-04.txt
print_period:[2]
Text name: res/res_cn_cls_cifar10_pairflip_rate45_lr1e-02_pi1_bias2e-01_k_mix10_gap_mixup08_sgd_max_epoch200_lsecoer1e-04.txt
print_period:[2]
Text name: res/res_cn_cls_cifar10_symmetric_rate50_lr1e-02_pi1_bias2e-01_k_mix10_gap_mixup04_sgd_max_epoch200_lsecoer1e-04.txt
print_period:[2]
Text name: res/res_cn_cls_cifar10_symmetric_rate20_lr1e-02_pi1_bias2e-01_k_mix10_gap_mixup08_sgd_max_epoch200_lsecoer1e-04.txt
print_period:[2]
Text name: res/res_cn_cls_cifar10_pairflip_rate45_lr1e-02_pi1_bias2